# Ускоренная оптимизация работы пункта выдачи заказов

## Оглавление
* [Описание проекта](#description)
* [Цель и задачи работы](#purpose)
* [Исходные данные](#init_data)
* [Импорт библиотек](#import)
* [Знакомство с данными](#data_info)
* [Предобработка данных](#preprocessing)
    * [Названия столбцов](#col_titles)
    * [Классификация заказов](#classification)
* [Исследовательский анализ данных](#investigation)
* [Общие выводы](#conclusion)

## Описание проекта <a class='anchor' id='description'></a>

Дан набор данных по продажам пункта выдачи заказов (ПВЗ) одного известного департмент стора. Его сотрудники столкнулись с проблемой высокой загрузки ПВЗ. Увеличение времени обслуживания клиентов приводит к ухудшению качества клиентского сервиса. Ранее выяснилось, что время ожидания на ПВЗ выросло из-за нехватки примерочных. Быстро произвести реорганизацию пространства ПВЗ и построить в 3 раза больше примерочных не видится возможным, а снизить количество недовольных клиентов требуется уже сейчас. Руководители ПВЗ высказали предположение, что постамат или наличие дополнительного места для примерки обуви — один из быстрых способов решения проблемы. Необходимо проверить это предположение на цифрах.

## Цель и задачи работы <a class='anchor' id='purpose'></a>

Целью настоящей работы была проверка гипотезы о том, что быстро снизить время ожидания клиентов можно за счет установки постамата или создания дополнительного места для примерки обуви.

Для достижения цели в работе решались следующие задачи:
* Предобработка данных;
* Классификация заказов в зависимости от содержимого и от количества позиций;
* Определение долей заказов, куда входит обувь или сумки, расчет количества позиций в таких заказах и вычисление их выкупаемости.

## Исходные данные <a class='anchor' id='init_data'></a>

* Week - неделя, на которой был совершен заказ
* Day - дата заказа
* DelivType - тип доставки
* PymentType - тип оплаты
* Journal_id - уникальный идентификатор заказа
* _ktt1 - категория товара 1го уровня вложенности
* _ktt2 - категория товара 2го уровня вложенности
* _ktt3 - категория товара 3го уровня вложенности
* _itemid - код товара
* _colorid - код цвета
* _Approved_USD - подтвержденный заказ в деньгах
* _Approved_PCS - подтвержденный заказ в штуках
* _Net_USD - выкупленный заказ в деньгах
* _Net_PCS - выкупленный заказ в штуках

## Импортируем библиотеки <a class='anchor' id='import'></a>

In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots 
from warnings import filterwarnings
filterwarnings('ignore')

## Знакомство с данными <a class='anchor' id='data_info'></a>

Загрузим датафрейм и посмотрим первые 5 строк.

In [ ]:
try:
    data = pd.read_excel('Тестовое задание - Анализ заказов ПВЗ (данные).xlsx')
except:
    data = pd.read_excel(
        'https://docs.google.com/spreadsheets/d/1Gfe520uN9bp3ccGJ7wBEy2x-8AefZSDe/edit?usp=drive_link&ouid=113002162786845116336&rtpof=true&sd=true'
        )
data.head()

Посмотрим общую информацию о датафрейме.

In [ ]:
data.info()

В датафрейме отсутствуют пропуски. Столбцы содержат данные нужного типа. В блоке по предобработке данных приведем названия столбцов к змеиному регистру и заменим название столбца `Day` на `date` для удобства.

Проверим наличие полных дубликатов.

In [ ]:
data.duplicated().sum()

В датафрейме отсутствуют полные дубликаты. Однако могут встречаться неявные дубликаты. Проверим их наличие. Посмотрим, есть ли дубли идентификаторов заказов.

In [ ]:
print(f'''Количество уникальных идентификаторов заказов = {data['Journal_id'].nunique()}
Общее количество идентификаторов заказов = {len(data)} ''')

Видим, что в датафрейме есть больше половины задублированных идентификаторов заказов.

Сохраним дубли и уникальные строки в отдельные датафреймы.

In [6]:
data_id_duplicates = data[data['Journal_id'].duplicated()]
data_without_id_duplicates = data.drop_duplicates(subset='Journal_id')

Выведем пример строк с задублированными идентификаторами.

In [ ]:
display(data_without_id_duplicates[data_without_id_duplicates['Journal_id'] == 
                                   data_id_duplicates['Journal_id'].reset_index(drop=True)[0]])
display(data_id_duplicates[data_id_duplicates['Journal_id'] == data_id_duplicates['Journal_id'].reset_index(drop=True)[0]])

Видно, что дубли, скорее всего, связаны с тем, что один и тот же клиент в одном заказе выбирает товары из разных категорий или товары из одинаковых категорий, но разных размеров/фасонов. Чтобы учесть каждую позицию и не записывать все в одну и ту же строку, сделаны дублирующие строки.

Таким образом, неявных дубликатов нет.

Посмотрим на распределения количественных признаков.

In [ ]:
fig = make_subplots(
    rows=2, 
    cols=2
)

fig.add_trace(go.Histogram(x=data['_Approved_PCS'],
                           xbins=dict(start=0, end=max(data['_Approved_PCS']), size=0.5),
                           name='Распределение подтвержденных заказов в штуках'),
              row=1,
              col=1,
              )

fig.add_trace(go.Histogram(x=data['_Net_PCS'],
                           xbins=dict(start=0, end=max(data['_Net_PCS']), size=0.5),
                           name='Распределение выкупленных заказов в штуках'),
              row=1,
              col=2,
              )

fig.add_trace(go.Histogram(x=data['_Approved_USD'],
                           xbins=dict(start=0, end=max(data['_Approved_USD']), size=100),
                           name='Распределение подтвержденных заказов в USD'),
              row=2,
              col=1,
              )

fig.add_trace(go.Histogram(x=data['_Net_USD'],
                           xbins=dict(start=0, end=max(data['_Net_USD']), size=300),
                           name='Распределение выкупленных заказов в USD'),
              row=2,
              col=2,
              )

fig.update_layout(height=500, 
                  width=1220, 
                  title_text="Распределения количественных признаков",
                  legend=dict(yanchor="top", y=1.35, xanchor="left", x=0.66))
fig.show()

В датафрейме присутствуют порядка 100 тыс. подтвержденных заказов, где ничего не заказали. Возможно, эти заказы относятся к категории `Услуги`. Проверим наше предположение.

In [ ]:
print(f'''Заказы, где нет выкупаемых позиций, относятся к категории \
"{data[(data['_Approved_USD'] == 0) & (data['_Approved_PCS'] == 0)]['_ktt2'].unique()[0]}"''')

Действительно, это `Услуги`. Проверим количество таких заказов.

In [ ]:
len(data[(data['_Approved_USD'] == 0) & (data['_Approved_PCS'] == 0)]['_ktt2'])

Результат совпадает с тем, что мы наблюдали на соответствующих гистограммах.

Также из построенных распределений видно следующее:
* В основном клиенты заказывают не более 1 вещи стоимостью не выше 2 тыс. USD. Есть отдельные заказы с большим количеством вещей и стоимостью повыше, но они составляют не более 10% от общего числа заказов. К выбросам их отнести нельзя, потому что такие заказы, действительно, имели место быть. Например, клиент хотел померить одежду или обувь разных размеров, чтобы затем выбрать подходящий вариант.
* Преобладают кейсы, когда клиент выкупает только 1 вещь или вовсе ничего. Сумма выкупаемых вещей находится в интервале до 1500 тыс. USD. Причем 70% приходится на покупки до 300 USD. Вероятно, причиной может быть то же самое, что и в предыдущем пункте.

Можно сделать следующие промежуточные выводы:
* В датафрейме нет пропусков и дубликатов как явных, так и неявных.
* Данные приведены к нужному типу.
* На этапе предобработки нужно привести названия столбцов к змеиному регистру и заменить название столбца `Day` на `date` для удобства.
* В датафрейме присутствуют порядка 100 тыс. подтвержденных заказов, где ничего не заказали. Они относятся к категории `Услуги`.

## Предобработка данных <a class='anchor' id='preprocessing'></a>

### Названия столбцов <a class='anchor' id='col_titles'></a>

Приведем названия столбцов к змеиному регистру и заменим название столбца `Day` на `date` для удобства.

Исходные названия столбцов следующие

In [ ]:
data.columns

In [12]:
data.columns = data.columns.str.lower()

После приведения к нижнему регистру заголовки имеют следующий вид

In [ ]:
data.columns

Внесем последние правки

In [14]:
data = data.rename(columns={
    'day': 'date',
    'delivtype': 'deliv_type',
    'pymenttype': 'payment_type',
    '_ktt1': 'ktt_1',
    '_ktt2': 'ktt_2',
    '_ktt3': 'ktt_3',
    '_itemid': 'item_id',
    '_colorid': 'color_id',
    '_approved_usd': 'approved_usd',
    '_approved_pcs': 'approved_pcs',
    '_net_usd': 'net_usd',
    '_net_pcs': 'net_pcs'
})

Итоговые названия столбцов теперь выглядят так

In [ ]:
data.columns

### Классификация заказов <a class='anchor' id='classification'></a>

Распределим заказы по следующим группам:
* нужна примерка в примерочной - все заказы, где хотя бы одна позиция одежда (ktt_2 = Одежда)
* легко примерить в зале - заказы, в которых только обувь и ничего больше (ktt_2 = Обувь). Заказы вида Обувь+Услуги подходят тоже
* примерка не нужна - для простоты возьмем заказы, в которых только сумки (ktt_2 = Сумки). Заказы вида Сумки+Услуги подходят тоже

Для таких целей лучше всего написать функцию.

In [16]:
def order_class(
        row: pd.Series
) -> str:
    '''
    Функция для классификации заказов по их содержимому
    '''

    try:
        if 'Одежда' in row or 'одежда' in row:
            return 'нужна примерка в примерочной'
        elif ('Обувь' in row or 'Обувь' in row and 'Услуги' in row) and \
            (
                not 'Бижутерия' in row and \
                not 'Аксессуары' in row and \
                not 'Сумки' in row and \
                not 'Подарочные карты' in row and \
                not 'Рюкзак дорожный' in row and \
                not 'Чемодан' in row   
            ):
            return 'легко примерить в зале'
        elif 'Сумки' in row and \
            (
                not 'Бижутерия' in row and \
                not 'Аксессуары' in row and \
                not 'Подарочные карты' in row and \
                not 'Рюкзак дорожный' in row and \
                not 'Чемодан' in row and \
                not 'Обувь' in row and \
                not 'Услуги' in row
            ):
            return 'примерка не нужна'
        else:
            return 'остальное'
    except:
        return 'нет категории'

Сгруппируем содержимое заказов `ktt_2` по id заказов.

In [ ]:
data_grouped_by_id = data.pivot_table(index='journal_id', values='ktt_2', aggfunc='sum').reset_index()
data_grouped_by_id.columns = ['journal_id', 'ktt_2']
data_grouped_by_id.head()

Применим написанную ранее функцию для классификации заказов по содержимому.

In [ ]:
data_grouped_by_id['category'] = data_grouped_by_id['ktt_2'].apply(order_class)
data_grouped_by_id.head(20)

Сделаем таблицу, где будут только уникальные идентификаторы заказов и их категория.

In [ ]:
data_grouped_by_id = data_grouped_by_id.drop('ktt_2', axis=1)
data_grouped_by_id.head()

## Исследовательский анализ данных <a class='anchor' id='investigation'></a>

**Посмотрим по месяцам, какую долю от общего числа заказов составляют заказы каждой из выделенных категорий.**

Сначала добавим к таблице с категориями заказов соответствующие месяцы из исходной таблицы.

In [ ]:
data_grouped_by_id_months = pd.merge(left=data_grouped_by_id, right=data.drop_duplicates(subset='journal_id'), on='journal_id', how='left')
data_grouped_by_id_months = data_grouped_by_id_months[['journal_id', 'category', 'date']]
data_grouped_by_id_months['date'] = data_grouped_by_id_months['date'].dt.month
data_grouped_by_id_months = data_grouped_by_id_months.rename(columns={'date': 'month'})
data_grouped_by_id_months.head()

Далее составим сводную таблицу, где помесячно рассмотрим количество заказов каждой категории.

In [ ]:
data_categories_share = data_grouped_by_id_months.pivot_table(
    index='month',
    columns='category',
    values='journal_id',
    aggfunc='count'
)
data_categories_share

Далее вычислим доли (в %) заказов каждой категории в общем пуле заказов.

In [ ]:
data_categories_share['total'] = data_categories_share.loc[:, 'легко примерить в зале':'примерка не нужна'].sum(axis=1)
data_categories_share['легко примерить в зале'] = data_categories_share['легко примерить в зале'] / data_categories_share['total'] * 100
data_categories_share['нужна примерка в примерочной'] = data_categories_share['нужна примерка в примерочной'] / data_categories_share['total'] * 100
data_categories_share['остальное'] = data_categories_share['остальное'] / data_categories_share['total'] * 100
data_categories_share['примерка не нужна'] = data_categories_share['примерка не нужна'] / data_categories_share['total'] * 100
data_categories_share = data_categories_share.drop('total', axis=1)
data_categories_share

Представим эту информацию также в графическом виде.

In [ ]:
fig = px.bar(
    data_frame=data_categories_share.reset_index().set_index('month'),
    title='Доля товаров разных категорий с разбивкой по месяцам',
    labels={'month': 'Месяцы', 'value': 'Доля, %'}
)
fig.update_layout(
    legend=dict(yanchor="top", y=1.45, xanchor="left", x=0.77),
    xaxis=dict(
        tickmode='array',
        tickvals=[i for i in range(1, 13)]
    )
    )
fig.show()

Промежуточные выводы:
* Заказы, где необходима примерка в примерочной (то есть присутствует какая-то одежда), ежемесячно составляют от 45 до 55 % от общего числа заказов. Месяцы, когда на долю таких заказов приходится менее 50%: март-май и сентябрь-декабрь - то есть сезоны `Весна` и `Осень`;
* На заказы, где примерка не нужна (только сумки) или товар можно легко примерить в зале (обувь или обувь+услуги) ежемесячно приходится 15 - 26 % от общего числа заказов. 
* Стоит обратить внимание на категорию `Остальное`. Их - от 19 до 39 % ежемесячно. Туда попали заказы, где есть различные аксессуары и ничего больше либо аксессуары и обувь с услугами. Для таких случаев можно предусмотреть в пункте место, куда можно класть получаемые аксессуары, чтобы их осмотреть. Если у клиента, помимо аксессуаров, будет еще и обувь, то он мог бы оставить аксессуары в отведенном месте и примерить обувь в зале. Кроме того, сотрудник, выдающий заказы, может приносить клиенту сначала обувь, чтобы тот померил ее в зале, а аксессуары - во вторую очередь.

**Посмотрим по месяцам количество заказов группы "Легко примерить в зале" по количеству заказанных позиций. Услуги исключим из расчета количества позиций**

Составим таблицу с количеством заказанных позиций в каждом уникальном заказе. Услуги не будем включать в расчет.

In [ ]:
data_counts = data[data['ktt_2'] != 'Услуги']['journal_id'].value_counts().reset_index()
data_counts.head()

Добавим данные о количестве заказанных позиций в таблицу с категориями и месяцами - `data_grouped_by_id_months` 

In [ ]:
data_grouped_by_id_months_count = pd.merge(left=data_grouped_by_id_months, right=data_counts, on='journal_id', how='left')
data_grouped_by_id_months_count.head()

Оставим только те заказы, которые относятся к категории `легко примерить в зале`

In [ ]:
data_positions_counts = data_grouped_by_id_months_count[
    data_grouped_by_id_months_count['category'] == 'легко примерить в зале'
    ]
data_positions_counts['count'] = data_positions_counts['count'].astype('int')
data_positions_counts.head()

Посмотрим, какие уникальные значения количества позиций в одном заказе встречаются в созданной таблице.

In [ ]:
sorted(data_positions_counts['count'].unique())

Напишем функцию для категоризации заказов из полученной таблицы по количеству товаров. Будем считать, что заказы, где больше 4 позиций (в данном случае - пар обуви), неудобны для примерки в зале, так как это будет захломлять зал и мешать другим клиентам. Поэтому отнесем такие заказы к категории `больше 4 позиций`.

In [28]:
def positions_counts_category(
        row: pd.Series
) -> str:
    try:
        if row == 1:
            return 'однотоварник'
        elif row == 2:
            return 'двухтоварник'
        elif row == 3:
            return 'трехтоварник'
        elif row == 4:
            return 'четырехтоварник'
        elif row > 5:
            return 'больше 4 позиций'
    except:
        return 'нет категории'

Применим написанную функцию для классификации заказов группы `легко примерить в зале` по количеству товаров

In [ ]:
data_positions_counts['counts_category'] = data_positions_counts['count'].apply(positions_counts_category)
data_positions_counts.head()

Сделаем сводную таблицу с количеством заказов в разрезе месяцев и категорий по числу позиций в заказе.

In [ ]:
data_positions_counts_pivot = (
    data_positions_counts
    .pivot_table(
        index='month',
        columns='counts_category',
        values='journal_id',
        aggfunc='count'
    )
    )
data_positions_counts_pivot

Представим эту таблицу также в графическом виде.

In [ ]:
fig = px.bar(
    data_frame=data_positions_counts_pivot.reset_index().set_index('month'),
    title='Количество заказов в зависимости от категории по числу позиций с разбивкой по месяцам',
    labels={'month': 'Месяцы', 'value': 'Количество заказов'}
)
fig.update_layout(
    legend=dict(yanchor="top", y=1.45, xanchor="left", x=0.85),
    xaxis=dict(
        tickmode='array',
        tickvals=[i for i in range(1, 13)]
    ),
    width=1200,
    height=600
    )
fig.show()

Промежуточные выводы:
* Среди заказов категории `легко примерить в зале` преобладают однотоварники - более 70% каждый месяц.
* Двухтоварников - от 10 до 20 %.
* На заказы с большим числом позиций приходится менее 5 - 10 %.

**Вычислим выкупаемость заказов из категории `легко примерить в зале`. Услуги в расчет также не берем.**

In [ ]:
data_net_to_approve = (
    pd.merge(
        left=data_positions_counts, 
        right=data[data['ktt_2'] != 'Услуги'][['journal_id', 'net_usd', 'approved_usd']], 
        on='journal_id', 
        how='left')
    )
data_net_to_approve.head()

Вычислим выкупаемость заказов.

In [ ]:
data_net_to_approve['net/approve'] = data_net_to_approve['net_usd'] / data_net_to_approve['approved_usd']
data_net_to_approve.head()

Сделаем сводную таблицу со значениями средней выкупаемости заказов категории `легко примерить в зале` в разрезе месяцев и типа заказа по числу позиций в нем.

In [ ]:
data_net_to_approve_pivot = (
    data_net_to_approve
    .pivot_table(
        index='month',
        columns='counts_category',
        values='net/approve',
        aggfunc='mean'
    )
)
data_net_to_approve_pivot

Представим ту же таблицу в графическом интерактивном виде.

In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    trace=go.Scatter(
        y=data_net_to_approve_pivot.reset_index()['однотоварник'],
        x=data_net_to_approve_pivot.reset_index()['month'],
        name='однотоварник'),
    row=1,
    col=1
),

fig.add_trace(
    trace=go.Scatter(
        y=data_net_to_approve_pivot.reset_index()['двухтоварник'],
        x=data_net_to_approve_pivot.reset_index()['month'],
        name='двухтоварник'),
    row=1,
    col=1
),

fig.add_trace(
    trace=go.Scatter(
        y=data_net_to_approve_pivot.reset_index()['трехтоварник'],
        x=data_net_to_approve_pivot.reset_index()['month'],
        name='трехтоварник'),
    row=1,
    col=1
),

fig.add_trace(
    trace=go.Scatter(
        y=data_net_to_approve_pivot.reset_index()['четырехтоварник'],
        x=data_net_to_approve_pivot.reset_index()['month'],
        name='четырехтоварник'),
    row=1,
    col=1
),

fig.add_trace(
    trace=go.Scatter(
        y=data_net_to_approve_pivot.reset_index()['больше 4 позиций'],
        x=data_net_to_approve_pivot.reset_index()['month'],
        name='больше 4 позиций'),
    row=1,
    col=1
)

fig.update_layout(
    legend=dict(yanchor="top", y=1.45, xanchor="left", x=0.85),
    xaxis=dict(
        tickmode='array',
        tickvals=[i for i in range(1, 13)]
    ),
    width=1200,
    height=600,
    xaxis_title = 'Месяцы',
    yaxis_title = 'Выкупаемость, доли ед.',
    title_text='Значения выкупаемости заказов по месяцам в зависимости от количества позиций'
    )
fig.show()

Видно, что выкупаемость для всех категорий заказов, которые легко примерить в зале, находится на уровне 0.48 - 0.53. То есть, как правило, клиент выкупает половину первоначального заказа. 

Заказы с 1 позицией показывают в 10 из 12 месяцев стабильную выкупаемость выше 0.49.

Заказы с 2 и 3 позициями показывают выкупаемость выше 0.49 только в 8 из 12 месяцев.

Заказы с 4 позициями показывают выкупаемость выше 0.49 только в 7 из 12 месяцев.

Заказы, содержащие больше 4 позиций, показывают выкупаемость выше 0.49 только в 6 из 12 месяцев.

## Общие выводы <a class='anchor' id='conclusion'></a>

Проанализировав заказы пункта выдачи за год, можно выделить следующие тренды:
* Заказы, где необходима примерка в примерочной (то есть присутствует какая-то одежда), ежемесячно составляют от 45 до 55 % от общего числа заказов. Месяцы, когда на долю таких заказов приходится более 50%: июнь-август и январь-февраль - то есть сезоны `Лето` и `Зима`

**=> Увеличить количество примерочных в будущем нужно. Лучше это сделать перед началом зимы или лета, когда больше половины заказов включают одежду**

* Стоит обратить внимание на заказы, где есть различные аксессуары и ничего больше либо аксессуары и обувь с услугами. Их - от 19 до 39 % ежемесячно. Для таких случаев можно предусмотреть в пункте место, куда можно класть получаемые аксессуары, чтобы их осмотреть. Если у клиента, помимо аксессуаров, будет еще и обувь, то он мог бы оставить аксессуары в отведенном месте и примерить обувь в зале. Кроме того, сотрудник, выдающий заказы, может приносить клиенту сначала обувь, чтобы тот померил ее в зале, а аксессуары - во вторую очередь.

**=> Предусмотреть в зале место, куда можно класть получаемые аксессуары, чтобы их осмотреть**

**=> При наличии в заказе аксессуаров и обуви лучше приносить клиенту сначала обувь, чтобы тот померил ее в зале, а аксессуары - во вторую очередь**

* На заказы, где примерка не нужна (только сумки) или товар можно легко примерить в зале (обувь или обувь+услуги) ежемесячно приходится 15 - 26 % от общего числа заказов. Причем основную долю составляют именно заказы с обувью.

**=> Для быстрой разгрузки пункта выдачи предусмотреть место для примерки обуви в зале - хорошее решение. Примерочные при этом могут быть разргружены почти на 25% каждый месяц. Ставить постамат будет менее эффективно, так как заказов с сумками не более 4% ежемесячно**

* Среди заказов с обувью и/или услугами преобладают однотоварники - более 70% каждый месяц. Двухтоварников - от 10 до 20 %. На заказы с большим числом позиций приходится менее 5 - 10 %.

**=> Клиентам будет удобно примерить обувь в зале, и зал не будет захломлен, так как в заказах с обувью обычно не более 1 - 2 позиций**

* Среднемесячная выкупаемость заказов с обувью находится на уровне 48-53% вне зависимости от количества позиций в заказе.

**=> Заказы с обувью показывают высокие значения выкупаемости, поэтому лучше обеспечить клиентам дополнительное место в зале для примерки, чтобы не снижать этот показатель**
